In [ ]:
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
from scipy import ndimage
import tensorflow as tf

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)): 
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\UC1_Nifti_Datasets_Paths\Datasets.json", 'r') as f:
    datasets = json.load(f)
prostatenet = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet"
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet\folds.json", 'r') as f:
    ValidationFolds = json.load(f)

In [ ]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_sitkobj()
patients_test = {key:patients[key] for key in patients.keys() if key in ValidationFolds["Fold 0"]}

In [ ]:
model_names = ['Attention_Unet', 'Dense2_Unet', 'Trans_Unet', 'Unet', 'USE_net', 'Vnet']
base_path = r"E:\DIMITRIS_DEEP_LEARNING\RESULTS\PROCANCER_UC1_RESULTS"
fold = "fold_0"
weights = "Final_1024.tf"
weights_check = "Checkpoint_1024.tf"


for md_nm in model_names:
    
    path_parent = [base_path, md_nm, fold]
    paths = [base_path, md_nm, fold, weights_check] # chains the paths
    model = tf.keras.models.load_model(os.path.join(*paths),compile = False)
    print(model.count_params())

    eval_params = {
        "MODEL": model,
        "EVALUATION_DATA": patients_test,# Needs to be a dictionary {PatName:{T2:arr, ADC:arr, DWI:arr, "Lesion":arr}}
        "SAVE_FOLDER":os.path.join(*path_parent) ,
    }

    eval = ProstateLesionDetectionUtils.Evaluation.Evaluation.Evaluation(eval_params)
    eval.predict()
    #eval.calculate_detorient_metric()
    eval.save_gt()
    eval.save_predictions()
    eval.save_predictions_binary()
    metrics = eval.get_metrics()
    preds = eval.get_predictions()
    
    for key,value in metrics.items():
        metrics[key][0].update({"Model": md_nm, "Number of Parameters":[model.count_params()]})
        
    path_parent.append("metrics_check.json")
    with open(os.path.join(*path_parent), "w") as f:
        json.dump(metrics, f, cls = NumpyEncoder, indent=4)